In [ ]:
import sys
from pathlib import Path

AVES_ROOT = Path("../..") if not "google.colab" in sys.modules else Path("aves_git")

EOD_PATH = AVES_ROOT / "data" / "external" / "EOD_STGO"
EOD_PATH

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import geopandas as gpd
from aves.data import eod, census
import matplotlib as mpl

# esto configura la calidad de la imagen. dependerá de tu resolución. el valor por omisión es 80
mpl.rcParams["figure.dpi"] = 150
# esto depende de las fuentes que tengas instaladas en el sistema.
mpl.rcParams["font.family"] = "Fira Sans Extra Condensed"

Cargamos las variables de persona `p`, hogar `h` y viajes `t`.

In [ ]:
p = eod.read_people(EOD_PATH)
h = eod.read_homes(EOD_PATH)
t = eod.read_trips(EOD_PATH)

In [ ]:
p.columns

In [ ]:
# descartamos sectores que no sean relevantes en los orígenes y destinos de los viajes
t = t[(t['SectorOrigen'] != 'Exterior a RM') 
        & (t['SectorDestino'] != 'Exterior a RM')
        & (t['SectorOrigen'] != 'Extensión Sur-Poniente') 
        & (t['SectorDestino'] != 'Extensión Sur-Poniente')
        & pd.notnull(t['SectorOrigen'])
        & pd.notnull(t['SectorDestino'])
        & pd.notnull(t['Proposito'])
]

print(len(t))

Antes considerábamos solamente la representatividad de los viajes en día de semana. Con esto incorporamos sábado y domingo. Para ello debemos considerar el peso o factor de expansión de cada viaje, y también el de cada persona. Por eso mezclamos ambos `DataFrame`:


In [ ]:
tp = t.merge(p)

In [ ]:
tp['trip_weight'] = tp['FactorLaboralNormal'].fillna(0) + tp['FactorSabadoNormal'].fillna(0) + tp['FactorDomingoNormal'].fillna(0)
tp['person_weight'] = tp['Factor_LaboralNormal'].fillna(0) + tp['Factor_SabadoNormal'].fillna(0) + tp['Factor_DomingoNormal'].fillna(0)
tp['weight'] = tp['trip_weight'] * tp['person_weight']
tp = tp[(tp['weight'] > 0)].copy()
tp['weight'].sum()

In [ ]:
tph = tp.merge(h)

Consideremos el contexto geográfico utilizando el censo y las áreas urbanas que calculamos en los notebooks anteriores.

In [ ]:
comunas = census.read_census_map('comuna', path=AVES_ROOT / "data" / "external" / "censo_2017_R13").to_crs('epsg:5361')
zones = gpd.read_file(AVES_ROOT / "data" / "processed" / "scl_zonas_urbanas.json").set_index('ID').to_crs(comunas.crs)

In [ ]:
from aves.features.geo import clip_area_geodataframe

In [ ]:
comunas_urbanas = (
    comunas[comunas["COMUNA"].isin(zones["Com"].unique())]
    .drop("NOM_COMUNA", axis=1)
    .copy()
)

comunas_urbanas["NombreComuna"] = comunas_urbanas["COMUNA"].map(
    dict(zip(zones["Com"], zones["Comuna"]))
)

comunas_urbanas.head()


In [ ]:
bounding_box = zones.total_bounds
comunas_urbanas = clip_area_geodataframe(comunas_urbanas, zones.total_bounds, buffer=1000)

In [ ]:
ax = comunas_urbanas.plot()
zones.plot(ax=ax, facecolor='none', edgecolor='black', linewidth=0.5)

## Vista General de la Movilidad en Santiago

In [ ]:
tph['Proposito'].value_counts()

In [ ]:
purposes_per_municipality = tph.groupby(['ComunaDestino', 'Proposito'])['trip_weight'].sum().unstack()
purposes_per_municipality

In [ ]:
purposes_per_municipality.loc['Santiago'].plot(kind='pie')

In [ ]:
purposes_per_municipality.loc['Providencia'].plot(kind='pie')

In [ ]:
trip_activities = {'Subsistencia': ['Al estudio', 'Al trabajo', 'Por trabajo', 'Por estudio'],
 'N/A': ['volver a casa'],
 'Mantención': ['De compras', 'Trámites', 'De salud'],
 'Discrecional': ['Buscar o Dejar a alguien',
  'Visitar a alguien',
  'Recreación',
  'Otra actividad (especifique)',
  'Comer o Tomar algo',
  'Buscar o dejar algo']}

In [ ]:
from aves.features.utils import normalize_rows

for key, cols in trip_activities.items():
    purposes_per_municipality[key] = purposes_per_municipality[cols].sum(axis=1)

activities_per_municipality = purposes_per_municipality[['Mantención', 'Subsistencia', 'Discrecional']].pipe(normalize_rows)
activities_per_municipality

In [ ]:
activities_per_municipality.loc[['Santiago', 'La Pintana']].T.plot(kind='pie', subplots=True)

In [ ]:
activities_per_municipality = gpd.GeoDataFrame(
    activities_per_municipality,
    geometry=comunas_urbanas.set_index("NombreComuna").buffer(-500).centroid,
)

activities_per_municipality

In [ ]:
from aves.visualization.figures import figure_from_geodataframe
from aves.visualization.colors import categorical_color_legend
import matplotlib.patheffects as path_effects

fig, ax = figure_from_geodataframe(zones, height=12)

comunas_urbanas.plot(ax=ax, edgecolor='white', facecolor='#efefef')

colors = sns.color_palette('cool', n_colors=3)

for idx, row in activities_per_municipality.iterrows():
    # posición en el espacio (coordenadas geográficas)
    pos = (row['geometry'].x, row['geometry'].y)
    # posición en el gráfico (coordenadas absolutas)
    p = ax.transData.transform_point(pos)
    # posición en la figura (coordenadas relativas)
    p = fig.transFigure.inverted().transform_point(p)

    pie_size = 0.03
    pie_bounds = [p[0] - pie_size * 0.5, p[1] - pie_size * 0.5, pie_size, pie_size]

    box_inset = fig.add_axes(pie_bounds, label=idx)
    box_inset.pie(row[['Mantención', 'Subsistencia', 'Discrecional']].values, wedgeprops=dict(edgecolor='black', linewidth=0.5), colors=colors)
    
    if True:
        pos_y = 1.0
        va = 'bottom'
        
        t = box_inset.annotate(idx, (0.5, pos_y), xycoords='axes fraction', 
                               horizontalalignment='center', va=va, fontsize=8, fontweight='bold', color='white')
        t.set_path_effects([path_effects.Stroke(linewidth=2, foreground='black'), path_effects.Normal()])
    
ax.set_title('Actividades Realizadas en Cada Comuna')


categorical_color_legend(ax, colors, ['Mantención', 'Subsistencia', 'Discrecional'], loc='center left')